# All improts necessary

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
%matplotlib inline

In [4]:
sns.set(font_scale=2)

# A few intro-words

# Read the data

In [5]:
data = pd.read_csv('../data/datasets/TS_Summer_2018/data.csv', sep=';')

In [6]:
data.head().T

,0,1,2,3,4
age,56,57,37,40,56
job,housemaid,services,services,admin.,services
marital,married,married,married,married,married
education,basic.4y,high.school,high.school,basic.6y,high.school
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
month,may,may,may,may,may
day_of_week,mon,mon,mon,mon,mon


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

# Add some gaps in the data

In [10]:
columns_with_gaps = data.columns[:-1]

In [16]:
columns_with_gaps

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')

In [32]:
minimum = 0
maximum = 0.3

In [33]:
columns_with_gaps_dict = dict(zip(columns_with_gaps, np.random.uniform(minimum, maximum, len(columns_with_gaps))))

In [34]:
columns_with_gaps_dict

{'age': 0.22143913505743587,
 'job': 0.005086069918096192,
 'marital': 0.07662325539614413,
 'education': 0.07961829451361443,
 'default': 0.015955954539032525,
 'housing': 0.06022691897093288,
 'loan': 0.24348751309850347,
 'contact': 0.1060436700035391,
 'month': 0.09808420148289115,
 'day_of_week': 0.1680493323326289,
 'duration': 0.029533894787779867,
 'campaign': 0.19197534957504975,
 'pdays': 0.06565918259465418,
 'previous': 0.03545157035973051,
 'poutcome': 0.24090895918204142,
 'emp.var.rate': 0.24792035598235665,
 'cons.price.idx': 0.2367022637369527,
 'cons.conf.idx': 0.16306440909804948,
 'euribor3m': 0.1791916814417334,
 'nr.employed': 0.11748529417688543}

In [15]:
data_with_gaps_v1 = data.copy()

In [35]:
for column in columns_with_gaps:
    if columns_with_gaps_dict[column] > 0:
        gaps_count = int(len(data_with_gaps_v1) * columns_with_gaps_dict[column])
        data_with_gaps_v1[column].iloc[np.random.randint(0, len(data_with_gaps_v1), gaps_count)] = None

In [37]:
data_with_gaps_v1.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               33045 non-null float64
job               40979 non-null object
marital           38144 non-null object
education         38021 non-null object
default           40539 non-null object
housing           38795 non-null object
loan              32272 non-null object
contact           37037 non-null object
month             37356 non-null object
day_of_week       34797 non-null object
duration          39984 non-null float64
campaign          34013 non-null float64
pdays             38571 non-null float64
previous          39756 non-null float64
poutcome          32366 non-null object
emp.var.rate      32111 non-null float64
cons.price.idx    32485 non-null float64
cons.conf.idx     35023 non-null float64
euribor3m         34421 non-null float64
nr.employed       36632 non-null float64
y                 41188 non-null object
dtypes: float64(10), object(11)
memory us

# Imputers

# Scalers

## Robust

## Non-robust

# Transformers

## Linear

## Non-linear

# Encoders

## One-Hot

## Mean

# Outliers

## By BoxPlot

## By MAD

# Target balancers

## Up-sampling

## Down-sampling

# Conclusion